Go through the GTF file and separate by protein coding genes (CDS, UTRs) and noncoding exons:

In [24]:
def find_gene_id(x):
	start = x.index('gene_id') + len('gene_id')+2
	start_to_end = x[start:]
	end = start_to_end.index(';')-1
	return start_to_end[:end]

def find_biotype(x):
	start = x.index('gene_biotype') + len('gene_biotype')+2
	start_to_end = x[start:]
	end = start_to_end.index(';')-1
	return start_to_end[:end]

with open('gtf_features.bed','w') as outfile:
    with open('/expanse/lustre/scratch/eschiksn/temp_project/IAN_MAPPING/genome_files/c_elegans.PRJNA13758.WS279.canonical_geneset.sorted.gtf') as gtf:
        for line in gtf:
            line=line.strip().split('\t')
            if line[2]=='exon':
                biotype=find_biotype(line[8])
                if biotype!='protein_coding':
                    chrom,start,end,strand,gene=line[0],str(int(line[3])-1),line[4],line[6],find_gene_id(line[8])
                    name='noncoding_exon_'+gene
                    out=[chrom,start,end,name,'60',strand]
                    outfile.write('\t'.join(out)+'\n')
            elif line[2]=='CDS' or 'utr' in line[2]:
                chrom,start,end,strand,gene=line[0],str(int(line[3])-1),line[4],line[6],find_gene_id(line[8])
                name=line[2]+'_'+gene
                out=[chrom,start,end,name,'60',strand]
                outfile.write('\t'.join(out)+'\n')

Convert intergenic saf to bed

In [25]:
with open('intergenic_regions.bed','w') as outfile:
    with open('/expanse/lustre/scratch/eschiksn/temp_project/IAN_MAPPING/genome_files/intergenic_regions.saf') as saf:
        for line in saf:
            line=line.strip().split()
            if line[0]=='GeneID':
                continue
            else:
                chrom,start,end,name=line[1],str(int(line[2])-1),line[3],line[0]
            out=[chrom,start,end,name,'60','+']
            outfile.write('\t'.join(out)+'\n')       

Combine introns, intergenic, gtf features

In [34]:
!echo "$(awk -F'\t' -vOFS='\t' '$4="intron_"$4' introns.bed)" > introns.bed

In [36]:
!cat gtf_features.bed introns.bed intergenic_regions.bed > combined_features.bed

Intersect with features

In [1]:
!bedtools intersect -a young_all_reps_AG_filtered.bed -b combined_features.bed -wb > young_all_reps_AG_FEATURES.bed

In [80]:
!wc -l young_all_reps_AG_FEATURES.bed

4485 young_all_reps_AG_FEATURES.bed


In [81]:
!bedtools intersect -a old_all_reps_AG_filtered.bed -b combined_features.bed -wb > old_all_reps_AG_FEATURES.bed

In [83]:
!wc -l old_all_reps_AG_FEATURES.bed

4657 old_all_reps_AG_FEATURES.bed


Because there are duplicates where one site overlaps multiple features (ie gene has multiple annotated UTRs, CDS that overlap), I need to filter this list to get it so that one site is only reported once. Also - need to account for strandedness for features that are not intergenic regions (intergenic regions don't require strandedness)

In [93]:
with open('young_all_reps_AG_FEATURES_tmp.bed','w') as outfile:
    with open('young_all_reps_AG_FEATURES.bed') as bed:
        for line in bed:
            line=line.strip().split()
            gene=line[9][line[9].rfind('_')+1:]
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            if feature != 'intergenic region' and line[5] != line[11]:
                continue 
            else:
                site_info=line[0:6]
                site_info.extend([gene,feature])
                outfile.write('\t'.join(site_info)+'\n')    

with open('old_all_reps_AG_FEATURES_tmp.bed','w') as outfile:
    with open('old_all_reps_AG_FEATURES.bed') as bed:
        for line in bed:
            line=line.strip().split()
            gene=line[9][line[9].rfind('_')+1:]
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            if feature != 'intergenic region' and line[5] != line[11]:
                continue 
            else:
                site_info=line[0:6]
                site_info.extend([gene,feature])
                outfile.write('\t'.join(site_info)+'\n')     

In [94]:
!wc -l young_all_reps_AG_FEATURES_tmp.bed # count lines just to see  - removes ~100 sites from wrong strand
!wc -l old_all_reps_AG_FEATURES_tmp.bed 

4338 young_all_reps_AG_FEATURES_tmp.bed
4500 old_all_reps_AG_FEATURES_tmp.bed


Keep only unique sites

In [95]:
!bedtools sort -i young_all_reps_AG_FEATURES_tmp.bed | uniq > young_all_reps_AG_FEATURES_final.bed
!bedtools sort -i old_all_reps_AG_FEATURES_tmp.bed | uniq > old_all_reps_AG_FEATURES_final.bed

In [96]:
!wc -l *FEATURES_final.bed

  4014 old_all_reps_AG_FEATURES_final.bed
  3911 young_all_reps_AG_FEATURES_final.bed
  7925 total


How many of these sites overlap with repetitive elements? Make a bed file of rmsk repetitive elements to intersect with edit beds. Get length of repeat elements to compare enrichment in edited sites as well

In [97]:
repeat_length=0
with open('repeats.bed','w') as outfile:
    with open('rmsk.txt') as repeats:
        for line in repeats:
            line=line.strip().split()
            chrom,start,end,name=line[5].replace('chr',''),line[6],line[7],line[10]
            out=[chrom,start,end,name,'60','+']
            repeat_length+=int(end)-int(start)
            outfile.write('\t'.join(out)+'\n')

Intersect repeat elements with edit sites

In [124]:
!bedtools intersect -a young_all_reps_AG_filtered.bed -b repeats.bed -wb > young_all_reps_AG_repeats.bed
!wc -l young_all_reps_AG_repeats.bed
!bedtools intersect -a old_all_reps_AG_filtered.bed -b repeats.bed -wb > old_all_reps_AG_repeats.bed
!wc -l old_all_reps_AG_repeats.bed

2677 young_all_reps_AG_repeats.bed
2715 old_all_reps_AG_repeats.bed


Remove duplicated entries

In [136]:
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' young_all_reps_AG_repeats.bed | bedtools sort | uniq > young_all_reps_AG_repeats_final.bed
!wc -l young_all_reps_AG_repeats_final.bed # looks good
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' old_all_reps_AG_repeats.bed | bedtools sort | uniq > old_all_reps_AG_repeats_final.bed
!wc -l old_all_reps_AG_repeats_final.bed # looks good

2569 young_all_reps_AG_repeats_final.bed
2576 old_all_reps_AG_repeats_final.bed


Intersect 3'UTR miRNA sites with edit sites

In [154]:
!bedtools intersect -a young_all_reps_AG_filtered.bed -b miR_site_6mer_coords.bed -wb > young_all_reps_AG_6mer.bed
!wc -l young_all_reps_AG_6mer.bed
!bedtools intersect -a old_all_reps_AG_filtered.bed -b miR_site_6mer_coords.bed -wb > old_all_reps_AG_6mer.bed
!wc -l old_all_reps_AG_6mer.bed
!bedtools intersect -a young_all_reps_AG_filtered.bed -b miR_site_7mer_coords.bed -wb > young_all_reps_AG_7mer.bed
!wc -l young_all_reps_AG_7mer.bed
!bedtools intersect -a old_all_reps_AG_filtered.bed -b miR_site_7mer_coords.bed -wb > old_all_reps_AG_7mer.bed
!wc -l old_all_reps_AG_7mer.bed

414 young_all_reps_AG_6mer.bed
545 old_all_reps_AG_6mer.bed
114 young_all_reps_AG_7mer.bed
138 old_all_reps_AG_7mer.bed


In this case it is helpful to know if edits overlap with specific miRNAs, so don't want to remove duplicates. But for the purposes of counting sites duplicates should not be considered multiple times.

In [155]:
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' young_all_reps_AG_6mer.bed | bedtools sort | uniq | wc -l
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' old_all_reps_AG_6mer.bed | bedtools sort | uniq | wc -l
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' young_all_reps_AG_7mer.bed | bedtools sort | uniq | wc -l
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' old_all_reps_AG_7mer.bed | bedtools sort | uniq | wc -l

382
495
109
133


Clean up files to save output

In [158]:
!awk '{print $1,$2,$3,$4,$5,$6.$10}' OFS='\t' young_all_reps_AG_6mer.bed | bedtools sort > young_all_reps_AG_6mer_final.bed
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' old_all_reps_AG_6mer.bed | bedtools sort > old_all_reps_AG_6mer_final.bed
!awk '{print $1,$2,$3,$4,$5,$6.$10}' OFS='\t' young_all_reps_AG_7mer.bed | bedtools sort > young_all_reps_AG_7mer_final.bed
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' old_all_reps_AG_7mer.bed | bedtools sort > old_all_reps_AG_7mer_final.bed

Now do all this with psU sites!

In [143]:
!wc -l young_psu_consistent_sites.bed
!wc -l old_psu_consistent_sites.bed

306 young_psu_consistent_sites.bed
472 old_psu_consistent_sites.bed


In [144]:
!bedtools intersect -a young_psu_consistent_sites.bed -b combined_features.bed -wb > young_all_reps_psu_FEATURES.bed
!bedtools intersect -a old_psu_consistent_sites.bed -b combined_features.bed -wb > old_all_reps_psu_FEATURES.bed

In [145]:
!wc -l young_all_reps_psu_FEATURES.bed
!wc -l old_all_reps_psu_FEATURES.bed

543 young_all_reps_psu_FEATURES.bed
860 old_all_reps_psu_FEATURES.bed


Because there are duplicates where one site overlaps multiple features (ie gene has multiple annotated UTRs, CDS that overlap), I need to filter this list to get it so that one site is only reported once. Also - need to account for strandedness for features that are not intergenic regions (intergenic regions don't require strandedness)

In [146]:
with open('young_all_reps_psu_FEATURES_tmp.bed','w') as outfile:
    with open('young_all_reps_psu_FEATURES.bed') as bed:
        for line in bed:
            line=line.strip().split()
            gene=line[9][line[9].rfind('_')+1:]
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            if feature != 'intergenic region' and line[5] != line[11]:
                continue 
            else:
                site_info=line[0:6]
                site_info.extend([gene,feature])
                outfile.write('\t'.join(site_info)+'\n')    

with open('old_all_reps_psu_FEATURES_tmp.bed','w') as outfile:
    with open('old_all_reps_psu_FEATURES.bed') as bed:
        for line in bed:
            line=line.strip().split()
            gene=line[9][line[9].rfind('_')+1:]
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            feature=line[9][:line[9].rfind('_')].replace('_',' ')
            if feature != 'intergenic region' and line[5] != line[11]:
                continue 
            else:
                site_info=line[0:6]
                site_info.extend([gene,feature])
                outfile.write('\t'.join(site_info)+'\n')     

In [147]:
!wc -l young_all_reps_psu_FEATURES_tmp.bed # count lines just to see  - removes ~100 sites from wrong strand
!wc -l old_all_reps_psu_FEATURES_tmp.bed 

521 young_all_reps_psu_FEATURES_tmp.bed
829 old_all_reps_psu_FEATURES_tmp.bed


Keep only unique sites

In [148]:
!bedtools sort -i young_all_reps_psu_FEATURES_tmp.bed | uniq > young_all_reps_psu_FEATURES_final.bed
!bedtools sort -i old_all_reps_psu_FEATURES_tmp.bed | uniq > old_all_reps_psu_FEATURES_final.bed

In [150]:
!wc -l *psu*FEATURES_final.bed

  488 old_all_reps_psu_FEATURES_final.bed
  318 young_all_reps_psu_FEATURES_final.bed
  806 total


How many of these sites overlap with repetitive elements? Make a bed file of rmsk repetitive elements to intersect with edit beds. Get length of repeat elements to compare enrichment in edited sites as well

Intersect repeat elements with edit sites

In [152]:
!bedtools intersect -a young_psu_consistent_sites.bed -b repeats.bed -wb > young_all_reps_psu_repeats.bed
!wc -l young_all_reps_psu_repeats.bed
!bedtools intersect -a old_psu_consistent_sites.bed -b repeats.bed -wb > old_all_reps_psu_repeats.bed
!wc -l old_all_reps_psu_repeats.bed

87 young_all_reps_psu_repeats.bed
110 old_all_reps_psu_repeats.bed


Remove duplicated entries

In [163]:
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' young_all_reps_psu_repeats.bed | bedtools sort | uniq > young_all_reps_psu_repeats_final.bed
!wc -l young_all_reps_psu_repeats_final.bed # looks good
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' old_all_reps_psu_repeats.bed | bedtools sort | uniq > old_all_reps_psu_repeats_final.bed
!wc -l old_all_reps_psu_repeats_final.bed # looks good

87 young_all_reps_psu_repeats_final.bed
110 old_all_reps_psu_repeats_final.bed


Intersect 3'UTR miRNA sites with edit sites

In [160]:
!bedtools intersect -a young_psu_consistent_sites.bed -b miR_site_6mer_coords.bed -wb > young_all_reps_psu_6mer.bed
!wc -l young_all_reps_psu_6mer.bed
!bedtools intersect -a old_psu_consistent_sites.bed -b miR_site_6mer_coords.bed -wb > old_all_reps_psu_6mer.bed
!wc -l old_all_reps_psu_6mer.bed
!bedtools intersect -a young_psu_consistent_sites.bed -b miR_site_7mer_coords.bed -wb > young_all_reps_psu_7mer.bed
!wc -l young_all_reps_psu_7mer.bed
!bedtools intersect -a old_psu_consistent_sites.bed -b miR_site_7mer_coords.bed -wb > old_all_reps_psu_7mer.bed
!wc -l old_all_reps_psu_7mer.bed

5 young_all_reps_psu_6mer.bed
14 old_all_reps_psu_6mer.bed
3 young_all_reps_psu_7mer.bed
3 old_all_reps_psu_7mer.bed


In this case it is helpful to know if edits overlap with specific miRNAs, so don't want to remove duplicates. But for the purposes of counting sites duplicates should not be considered multiple times.

In [161]:
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' young_all_reps_psu_6mer.bed | bedtools sort | uniq | wc -l
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' old_all_reps_psu_6mer.bed | bedtools sort | uniq | wc -l
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' young_all_reps_psu_7mer.bed | bedtools sort | uniq | wc -l
!awk '{print $1,$2,$3,$4,$5,$6}' OFS='\t' old_all_reps_psu_7mer.bed | bedtools sort | uniq | wc -l

4
13
3
3


Clean up files to save output

In [162]:
!awk '{print $1,$2,$3,$4,$5,$6.$10}' OFS='\t' young_all_reps_psu_6mer.bed | bedtools sort > young_all_reps_psu_6mer_final.bed
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' old_all_reps_psu_6mer.bed | bedtools sort > old_all_reps_psu_6mer_final.bed
!awk '{print $1,$2,$3,$4,$5,$6.$10}' OFS='\t' young_all_reps_psu_7mer.bed | bedtools sort > young_all_reps_psu_7mer_final.bed
!awk '{print $1,$2,$3,$4,$5,$6,$10}' OFS='\t' old_all_reps_psu_7mer.bed | bedtools sort > old_all_reps_psu_7mer_final.bed

In [164]:
!pwd

/expanse/lustre/scratch/eschiksn/temp_project/IAN_MAPPING/RNA_mods


In [165]:
!ls *AG*final*

old_all_reps_AG_6mer_final.bed	    young_all_reps_AG_6mer_final.bed
old_all_reps_AG_7mer_final.bed	    young_all_reps_AG_7mer_final.bed
old_all_reps_AG_FEATURES_final.bed  young_all_reps_AG_FEATURES_final.bed
old_all_reps_AG_repeats_final.bed   young_all_reps_AG_repeats_final.bed
